In [ ]:
import torch
import torch.nn as nn
from transformers import (
    InstructBlipForConditionalGeneration,
    BlipImageProcessor,
    AutoTokenizer,
    AutoModelForCausalLM,
    InstructBlipQFormerConfig,
    InstructBlipQFormerModel
)
from PIL import Image
import pandas as pd
import os
import re
from tqdm.auto import tqdm
from torch.cuda.amp import autocast
import random
import numpy as np
import matplotlib.pyplot as plt

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# --- 기본 설정 ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16

In [ ]:
# --- SFT 모델과 동일한 아키텍처 재현 ---
print("--- 추론 모델 아키텍처 로드 ---")

# 베이스 모델 로드 (컴포넌트 추출용)
print("1) 베이스 모델 로드 (Hugging Face Hub)")

base_blip_model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-flan-t5-xl", low_cpu_mem_usage=True
)
base_llm_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat", trust_remote_code=True, low_cpu_mem_usage=True
)
llm_tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen-1_8B-Chat", trust_remote_code=True
)
image_processor = BlipImageProcessor.from_pretrained(
    "Salesforce/instructblip-flan-t5-xl"
)

# SFT 모델과 동일한 컴포넌트 생성
print("2) share_instructblip_qwen 컴포넌트 생성 중...")
vision_model = base_blip_model.vision_model
query_tokens = base_blip_model.query_tokens
qformer_config = base_blip_model.qformer.config
qformer_config.num_hidden_layers = 10 # SFT 때와 동일하게 10개 레이어
shrunken_qformer = InstructBlipQFormerModel(qformer_config)
llm_model = base_llm_model
image_proj = nn.Linear(shrunken_qformer.config.hidden_size, llm_model.config.hidden_size)
print("모델 로드 완료!")

del base_blip_model, base_llm_model

In [109]:
# SFT 모델과 동일한 최종 모델 클래스 정의
class VisionLLM_QFormer(nn.Module):
    def __init__(self, vision_model, qformer, query_tokens, image_proj, llm_model):
        super().__init__()
        self.vision_model = vision_model
        self.qformer = qformer
        self.image_proj = image_proj
        self.llm_model = llm_model
        self.query_tokens = query_tokens

    def forward(self, pixel_values, input_ids, attention_mask, labels=None):
        # 학습 시에만 사용
        pass


In [ ]:
# 최종 모델 껍데기 생성
inference_model = VisionLLM_QFormer(
    vision_model, shrunken_qformer, query_tokens, image_proj, llm_model
).to(device)

print("추론 모델 아키텍처 재현 완료.")


# --- 학습된 SFT 가중치 로드 ---
SFT_MODEL_PATH = "share_instructblip_qwen_sft.pth"
if os.path.exists(SFT_MODEL_PATH):
    print(f"\n학습된 SFT 모델 가중치를 로드합니다: {SFT_MODEL_PATH}")
    # state_dict를 로드하고, 모델을 dtype과 device에 맞게 설정
    inference_model.load_state_dict(torch.load(SFT_MODEL_PATH, map_location=device))
    inference_model.to(dtype=dtype)
else:
    raise FileNotFoundError(f"오류: SFT 모델 파일({SFT_MODEL_PATH})을 찾을 수 없습니다. 추론을 진행할 수 없습니다.")

# 모델을 추론 모드로 설정
inference_model.eval()
print("모델을 추론 모드(eval)로 설정했습니다.")

# Qwen 토크나이저의 pad_token 설정
if llm_tokenizer.pad_token is None:
    llm_tokenizer.pad_token = llm_tokenizer.eos_token


In [111]:
# === 추론 함수 정의 ===
@torch.no_grad()
def multimodal_greedy_generate(
    model, full_embeds, attention_mask,
    max_new_tokens=64, eos_token_id=None
):
    generated_ids = []
    cur_embeds = full_embeds
    cur_attention_mask = attention_mask
    batch_size = cur_embeds.size(0)
    
    for _ in range(max_new_tokens):
        # 모델의 LLM 부분만 사용하여 다음 토큰 예측
        outputs = model.llm_model(
            inputs_embeds=cur_embeds,
            attention_mask=cur_attention_mask,
            use_cache=True, # 추론 속도 향상을 위해 use_cache 활성화
            return_dict=True
        )
        next_token_logits = outputs.logits[:, -1, :]
        next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
        generated_ids.append(next_token_id)
        
        if eos_token_id is not None and (next_token_id.squeeze(1) == eos_token_id).all():
            break

        # 다음 입력을 위해 생성된 토큰의 임베딩을 현재 임베딩에 추가
        next_embeds = model.llm_model.get_input_embeddings()(next_token_id)
        cur_embeds = torch.cat([cur_embeds, next_embeds], dim=1)
        # 어텐션 마스크도 1 추가
        cur_attention_mask = torch.cat(
            [cur_attention_mask, torch.ones((batch_size, 1), dtype=cur_attention_mask.dtype, device=device)], dim=1
        )

    return torch.cat(generated_ids, dim=1) if generated_ids else torch.tensor([[]], device=device, dtype=torch.long)

prompt_template = """You are a helpful visual assistant.  Based on the image, answer the following multiple-choice question by providing only:

- The letter (A, B, C, or D) of the correct option on the first line
- A brief reason for your choice on the second line

Here is an example:

Question: What causes the sharp pain often felt in the head when eating something like this too quickly?

Options:
(A) Freezing of taste buds on the tongue	
(B) Sudden brain pressure due to rapid intake	
(C) Ice touching the tongue causing a shock	
(D) Blood vessel reaction in the palate causing discomfort

The letter of the correct answer is: D
Reasoning: The model selects this because the sharp pain from eating cold foods is medically known to result from rapid constriction and dilation of blood vessels in the palate.

- The letter (A, B, C, or D) of the correct option on the first line
- A brief reason for your choice on the second line

Now, answer the following question:

Question: {question}

Options:
(A) {option_A}
(B) {option_B}
(C) {option_C}
(D) {option_D}

The letter of the correct answer is: """

def get_best_option_by_manual_prompting(model, processor, tokenizer, image_path, question, options, prompt_template, device):
    try:
        raw_image = Image.open(image_path).convert('RGB')
        pixel_values = processor(images=raw_image, return_tensors="pt")["pixel_values"].to(device, dtype=dtype)
    except Exception as e:
        # print(f"이미지 로드 실패: {image_path}, 에러: {e}")
        return None

    full_prompt = prompt_template.format(
        question=question,
        option_A=options.get('A', ''), option_B=options.get('B', ''),
        option_C=options.get('C', ''), option_D=options.get('D', '')
    )
    
    inputs = tokenizer(full_prompt, return_tensors='pt').to(device)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    with torch.no_grad(), autocast(dtype=dtype):
        # Vision-Language 임베딩 생성 (학습 코드와 동일한 로직)
        image_embeds = model.vision_model(pixel_values=pixel_values, return_dict=True).last_hidden_state
        image_attention_mask = torch.ones(image_embeds.shape[:-1], dtype=torch.long, device=device)
        
        expanded_query_tokens = model.query_tokens.expand(image_embeds.shape[0], -1, -1)
        query_shape = expanded_query_tokens.shape
        dummy_query_input_ids = torch.zeros((query_shape[0], query_shape[1]), dtype=torch.long, device=device)
        
        query_outputs = model.qformer(
            input_ids=dummy_query_input_ids,
            query_embeds=expanded_query_tokens,
            encoder_hidden_states=image_embeds,
            encoder_attention_mask=image_attention_mask,
            return_dict=True
        ).last_hidden_state
        
        projected_feats = model.image_proj(query_outputs)
        input_embeds = model.llm_model.get_input_embeddings()(input_ids)
        
        # 최종 임베딩과 어텐션 마스크 결합
        full_embeds = torch.cat([projected_feats, input_embeds], dim=1)
        full_attention_mask = torch.cat([torch.ones(projected_feats.shape[:2], device=device), attention_mask], dim=1)

        # 텍스트 생성
        generated_ids = multimodal_greedy_generate(
            model=model,
            full_embeds=full_embeds,
            attention_mask=full_attention_mask,
            max_new_tokens=5, # 정답 옵션(A,B,C,D)과 약간의 설명을 포함할 수 있도록 길이 조절
            eos_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()
    
    # 생성된 텍스트에서 가장 먼저 나오는 알파벳(A,B,C,D)을 정답으로 파싱
    match = re.search(r'\b([A-D])\b', generated_text.upper())
    if match:
        return match.group(1)
    else:
        # print(f"경고: 유효한 선택지(A,B,C,D)를 생성하지 못했습니다. 모델 출력: '{generated_text}'")
        return None

In [ ]:
# === 추론 실행 및 제출 파일 생성 ===
print("\ntest.csv 데이터로 추론 및 제출 파일 생성을 시작합니다...")

# CSV 파일 및 이미지 경로 설정 - 상대경로로 지정
test_csv_path = "./open/test.csv"
submission_csv_path = "./open/sample_submission.csv"
test_img_base_path = "./open/test_input_images"

try:
    test_df = pd.read_csv(test_csv_path)
    submission_df = pd.read_csv(submission_csv_path)

    predictions = []

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inferencing for submission"):
        img_filename = row['ID'] + '.jpg'
        img_path = os.path.join(test_img_base_path, img_filename)
        
        if not os.path.exists(img_path):
            predictions.append('B')
            continue

        question = row['Question']
        options = {'A': row['A'], 'B': row['B'], 'C': row['C'], 'D': row['D']}
        
        predicted_answer = get_best_option_by_manual_prompting(
            inference_model, image_processor, llm_tokenizer,
            img_path, question, options, prompt_template, device
        )

        if predicted_answer is None:
            predicted_answer = 'B'
        
        predictions.append(predicted_answer)

    if len(predictions) == len(submission_df):
        submission_df['answer'] = predictions
        output_submission_path = "./open/submission.csv"
        submission_df.to_csv(output_submission_path, index=False)
        print("\n" + "="*50)
        print("추론 및 제출 파일 생성 완료 🎉")
        print(f"  - 총 {len(predictions)}개의 예측을 완료했습니다.")
        print(f"  - 제출 파일이 '{output_submission_path}' 경로에 저장되었습니다.")
        print("="*50)
    else:
        print(f"\n오류: 예측된 결과의 수({len(predictions)})와 제출 파일의 행 수({len(submission_df)})가 일치하지 않습니다.")

except FileNotFoundError as e:
    print(f"제출 파일 생성 오류: 필수 파일({e.filename})을 찾을 수 없습니다.")

In [ ]:
# === train.csv를 이용한 정확도 평가 ===
print("\ntrain.csv 데이터로 정확도 평가를 시작합니다...")

# CSV 파일 및 이미지 경로 설정 - 상대경로로 지정됨
train_csv_path = "./open/train.csv"
train_img_base_path = "./open/train_input_images"

try:
    train_df = pd.read_csv(train_csv_path)
except FileNotFoundError:
    print(f"평가 오류: train.csv 파일을 '{train_csv_path}' 경로에서 찾을 수 없습니다.")
    train_df = None

if train_df is not None:
    correct_predictions = 0
    total_valid_samples = 0

    for index, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Evaluating"):
        img_filename = row['ID'] + '.jpg'
        img_path = os.path.join(train_img_base_path, img_filename)
        
        if not os.path.exists(img_path):
            continue # 이미지가 없으면 건너뛰기

        question = row['Question']
        options = {'A': row['A'], 'B': row['B'], 'C': row['C'], 'D': row['D']}
        true_answer = row['answer']
        
        # 모델을 통해 가장 좋은 선택지 예측
        predicted_answer = get_best_option_by_manual_prompting(
            inference_model,
            image_processor,
            llm_tokenizer,
            img_path,
            question,
            options,
            prompt_template,
            device
        )

        print(predicted_answer)

        if predicted_answer is None:
            continue # 모델이 유효한 답변을 생성하지 못하면 건너뛰기
        
        total_valid_samples += 1
        if predicted_answer == true_answer:
            correct_predictions += 1

    # 최종 평가 결과 출력
    accuracy = (correct_predictions / total_valid_samples) * 100 if total_valid_samples > 0 else 0
    print("\n" + "="*50)
    print("평가 완료")
    print(f"  - 전체 유효 샘플 수: {total_valid_samples}")
    print(f"  - 정답 개수: {correct_predictions}")
    print(f"  - 정확도 (Accuracy): {accuracy:.2f}%")
    print("="*50)